# Machine Learning in Python - Keras & Scikit-Learn

In [ ]:
import pandas as pd
import numpy as np

from IPython.display import display, HTML, Image
from IPython.display import SVG

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('fivethirtyeight')

from TAS_Python_Utilities import data_viz
from TAS_Python_Utilities import visualize_tree

from sklearn.tree import export_graphviz
from sklearn import metrics
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn import preprocessing 

from keras.models import Sequential
from keras.layers import Dense, Activation, Merge, Dropout
from keras.utils.np_utils import to_categorical
from keras.utils.vis_utils import model_to_dot
from keras.wrappers.scikit_learn import KerasClassifier
 

## Dataset Pre-processing

Read in a dataset

In [ ]:
abt = pd.read_csv("mnist_train_small.csv", encoding = "ISO-8859-1")

# Put all but the target variable into the descriptive features array
X = abt[abt.columns.difference(["value"])]
Y = abt["value"]

# Use a range scaling to scale all variables to between 0 and 1
min_max_scaler = preprocessing.MinMaxScaler()
cols = X.columns
X = pd.DataFrame(min_max_scaler.fit_transform(X), columns = cols) # Watch out for putting back in columns here

X_train_plus_valid, X_test, y_train_plus_valid, y_test = train_test_split(X, Y, random_state=0, test_size = 0.30, train_size = 0.7)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_plus_valid, y_train_plus_valid, random_state=0, test_size = 0.199/0.7, train_size = 0.5/0.7)

# Convert the singl column label into a dummy coded label
y_train_wide = to_categorical(np.asarray(y_train))
y_train_plus_valid_wide = to_categorical(np.asarray(y_train_plus_valid))
y_valid_wide = to_categorical(np.asarray(y_valid))

## Model Tuning

Specfiy the structure of the neural network model and training parameters in a function

In [ ]:
# Function to create model, required for KerasClassifier
def create_model(optimiser = "rmsprop", hidden_units = 512):
    # create model
    model = Sequential()
    model.add(Dense(input_dim=784, units=hidden_units))
    model.add(Activation("sigmoid"))
    model.add(Dense(units=10))
    model.add(Activation("softmax"))
    # Compile model
    model.compile(optimizer=optimiser,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
    return model


Perform a grid search

In [ ]:
param_grid ={'optimiser': ['rmsprop', 'adam'], \
             'hidden_units':[100, 200, 300],
#            'epochs': [20, 50, 100, 150], \
              'epochs': [4, 8], \
#             'batch_size': [8, 16, 32, 64]}
             'batch_size': [8, 16]}
 
model = KerasClassifier(build_fn=create_model, verbose=0)
my_tuned_model = GridSearchCV(estimator=model, param_grid=param_grid, verbose = 2, cv = 2)
my_tuned_model.fit(np.asfarray(X_train_plus_valid), np.asfarray(y_train_plus_valid))

# summarize results
print("Best: %f using %s" % (my_tuned_model.best_score_, my_tuned_model.best_params_))
means = my_tuned_model.cv_results_['mean_test_score']
stds = my_tuned_model.cv_results_['std_test_score']
params = my_tuned_model.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))
    

Draw the model

In [ ]:
SVG(model_to_dot(my_tuned_model).create(prog='dot', format='svg'))

Evaluate the model on a test dataset

In [ ]:
print("****** Test Data ********")

# Make a set of predictions for the validation data
y_pred = my_tuned_model.predict(np.asfarray(X_test))

# Print performance details
print(metrics.classification_report(y_test, y_pred))

# Print confusion matrix
print("Confusion Matrix")
display(pd.crosstab(y_test, y_pred, rownames=['True'], colnames=['Predicted'], margins=True))